# Connect to google account

In [ ]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount("/content/data")

Mounted at /content/data


In [ ]:
# 모듈 Import
import random
import numpy as np # 행렬 계산에 사용하는 모듈
import pandas as pd # 데이터 처리와 분석을 위한 모듈
import matplotlib.pyplot as plt # 데이터 시각화를 위한 모듈. 2D, 3D 그릴 때 사용
import seaborn as sns # 데이터 시각화를 위한 모듈. 두 데이터의 관계를 볼때 사용
from tqdm.auto import tqdm
from collections import defaultdict
from sklearn.decomposition import TruncatedSVD, NMF, SparsePCA
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# 딕셔너리를 간편하게 사용하는 것을 도와준다. 모델 class 처럼 사용 가능
# dot을 이용해 객체를 불러 사용. JSON 다룰때 유용.
import easydict
args = easydict.EasyDict()

# path
args.default_path = "/content/data/MyDrive/Playdata/Competitions/ML/Dacon/" # 메인 경로
args.apply_train_path = args.default_path + "apply_train.csv" # train 데이터 경로
args.company_path = args.default_path + "company.csv"
args.recruitment_path = args.default_path + "recruitment.csv"
args.resume_certificate_path = args.default_path + "resume_certificate.csv"
args.resume_education_path = args.default_path + "resume_education.csv"
args.resume_language_path = args.default_path + "resume_language.csv"
args.resume_path = args.default_path + "resume.csv"

args.default_submission_path = args.default_path + "sample_submission.csv" # 예측결과(제출파일) 경로

# 데이터 분석을 위한 변수들
# # 난수 생성 제어 => 같은 코드를 실행해도 동일한 결과를 얻기 위해서 설정
# 데이터 분할 및 모델 초기화 때 유용.
args.random_state = 42
args.results = [] # 결과 저장 리스트

In [ ]:
args.submission_path = args.default_path + "result/submission_231106.csv" # 결과 저장 파일
args.save_results = args.default_path+"result/model_results_231106.json" # 결과 저장 json

# File Load

In [ ]:
apply_train_df = pd.read_csv(args.apply_train_path) # apply_train -> DataFrame화
company_df = pd.read_csv(args.company_path) # company -> DataFrame화
recruitment_df = pd.read_csv(args.recruitment_path) # recruitment -> DataFrame화
resume_certificate_df = pd.read_csv(args.resume_certificate_path) # resume_certificate -> DataFreame화
resume_education_df = pd.read_csv(args.resume_education_path) # resume_education -> DataFrame화
resume_language_df = pd.read_csv(args.resume_language_path) # resume_language -> DataFrame화
resume_df = pd.read_csv(args.resume_path) # resume -> DataFrame화


In [ ]:
apply_train = apply_train_df.copy()
company = company_df.copy()
recruitment = recruitment_df.copy()
resume_certificate = resume_certificate_df.copy()
resume_education = resume_education_df.copy()
resume_language = resume_language_df.copy()
resume = resume_df.copy()

# 탐색

### merged_recruitment 생성

In [ ]:
# recruitment, company 정렬(recruitment_seq 기준)
company = company.sort_values(by = "recruitment_seq")
recruitment = recruitment.sort_values(by = "recruitment_seq")

In [ ]:
# merged_recruitment = recruitment, company merge
merged_recruitment = pd.merge(recruitment, company, on='recruitment_seq', how='left')
merged_recruitment

,recruitment_seq,address_seq1,address_seq2,address_seq3,career_end,career_start,check_box_keyword,education,major_task,qualifications,text_keyword,company_type_seq,supply_kind,employee
0,R00001,5.0,NaN,NaN,0,0,2101;2108;2201;2204;2205;2707;2810,2,2,1,NaN,5.0,201.0,631.0
1,R00002,3.0,NaN,NaN,0,0,2507;2703;2707,3,2,1,NaN,2.0,201.0,160.0
2,R00003,3.0,NaN,NaN,0,0,2101;2108;2201;2707,3,2,2,NaN,NaN,NaN,NaN
3,R00004,3.0,NaN,NaN,0,0,2507;2707,3,2,1,NaN,2.0,402.0,500.0
4,R00005,3.0,NaN,NaN,0,0,2507;2707,3,2,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6690,R06691,3.0,NaN,NaN,0,0,2501;2507;2707,3,2,1,NaN,NaN,NaN,NaN
6691,R06692,3.0,NaN,NaN,0,0,2201;2507,3,2,2,NaN,4.0,402.0,150.0
6692,R06693,5.0,NaN,NaN,0,0,2102;2707,4,2,1,NaN,NaN,NaN,NaN
6693,R06694,3.0,NaN,NaN,0,0,2101;2108;2109;2110;2201;2203;2707,4,2,1,NaN,NaN,NaN,NaN


### merged_recruitment 탐색 & drop

In [ ]:
merged_recruitment["text_keyword"][:100]

0                        NaN
1                        NaN
2                        NaN
3                        NaN
4                        NaN
               ...          
95    영업팀장;클리지;영업부서장;영업;영업총괄
96                       NaN
97                       NaN
98                       NaN
99                       NaN
Name: text_keyword, Length: 100, dtype: object

In [ ]:
merged_recruitment.info()

In [ ]:
merged_recruitment.describe()

In [ ]:
(merged_recruitment.isnull().sum() / len(merged_recruitment)).sort_values()

- 결측치가 90프로 이상인 컬럼 2개 존재
  - address_seq2, address_seq3 : drop 해도 괜찮을 것 같다.
- text_keyword(모집직무키워드) : 키워드 정리할 수 있는 방법
- employee(종업원수), supply_kind(주업종코드), company_type_seq(회사유형코드) : 동일한 index의 값이 결측치다. 분포도를 확인하여 채울 것인지 drop 할건지 결정
- address_seq1(근무지주소) : mode로 해도 될듯하다.

[other col]
- career_start, career_end : 값이 다 없는 상황이기에 drop 해도 괜찮을 것 같다.
- education : 요구학위와 resume의 최종학력과의 관계?
- company_type_seq : 중소, 중견, 대기업 구분?

In [ ]:
merged_recruitment_drop_col = ['address_seq2','address_seq3', 'career_start', 'career_end']
merged_recruitment = merged_recruitment.drop(merged_recruitment_drop_col, axis=1)
merged_recruitment.info()

- 결측치
  - address_seq1 : mode

In [ ]:
addr_mode = merged_recruitment["address_seq1"].mode().values[0]
merged_recruitment["address_seq1"].fillna(addr_mode, inplace = True)
merged_recruitment.info()

- 결측치
  - company_type_seq : mode
  - supply_kind, employee : company_type_seq mode값에 맞게 결정

In [ ]:
sns.displot(merged_recruitment, x = "company_type_seq")

In [ ]:
merged_recruitment["company_type_seq"].value_counts()

In [ ]:
company_type_seq_mode = merged_recruitment["company_type_seq"].mode().values[0]
merged_recruitment["company_type_seq"].fillna(company_type_seq_mode, inplace = True)
merged_recruitment.info()

In [ ]:
company_list = merged_recruitment.loc[merged_recruitment["company_type_seq"] == company_type_seq_mode, "supply_kind"].dropna().values
supply_mode = pd.DataFrame(company_list).mode()
supply_mode
supply_mode.values[0][0]

In [ ]:
merged_recruitment["supply_kind"].fillna(supply_mode.values[0][0], inplace = True)
merged_recruitment.info()

In [ ]:
company_em_list = merged_recruitment.loc[merged_recruitment["company_type_seq"] == company_type_seq_mode, "employee"].dropna().values
employee_mode = pd.DataFrame(company_em_list).mode()
employee_mode
employee_mode.values[0][0]

In [ ]:
merged_recruitment["employee"].fillna(employee_mode.values[0][0], inplace = True)
merged_recruitment.info()

In [ ]:
fillna_col = ['text_keyword']

for col in fillna_col: #for문을 돌면서 각 컬럼에 대한 최빈값으로 채움
  col_mode = merged_recruitment[col].mode().values[0]
  merged_recruitment[col].fillna(col_mode, inplace=True)

(merged_recruitment.isnull().sum() / len(merged_recruitment)).sort_values()

- 중소 / 중소 외 나누기
  - 중소 1 / 중소 외 2

In [ ]:
merged_recruitment["company_type_seq"].unique()

In [ ]:
company_standard = merged_recruitment["company_type_seq"] == 2.0
merged_recruitment.loc[company_standard, "company_standard"]= 1
merged_recruitment["company_standard"].fillna(2.0, inplace = True)
merged_recruitment["company_standard"] = merged_recruitment["company_standard"].astype('int')
merged_recruitment["company_standard"] = merged_recruitment["company_standard"].astype('category')
merged_recruitment.head()

In [ ]:
#문자형(object type)을 LightGBM에 사용할 수 있도록 Category 형태로 형 변환
merged_recruitment_object = merged_recruitment.select_dtypes(exclude=np.number)
object_cols = list(merged_recruitment_object.columns)

for col in object_cols:
  merged_recruitment[col] = merged_recruitment[col].astype('category')

#형 변환 확인
merged_recruitment.info()

### merged_resume 생성

In [ ]:
# resume 정렬 (resume_seq 기준)
resume = resume.sort_values(by='resume_seq')

In [ ]:
# resume_certificate 정렬 (resume_seq 기준)
resume_certificate = resume_certificate.sort_values(by='resume_seq')
# certificate_contents에 결측값이 존재하는 행 제거
resume_certificate = resume_certificate.dropna(subset=['certificate_contents'])
# 이력서번호(resume_seq)기준으로 groupby
# 이력서번호(resume_seq)의 해당하는 자격증들을 세미콜론(;)으로 join해서 적용(apply)
resume_certificate = resume_certificate.groupby('resume_seq')['certificate_contents'].apply(';'.join).reset_index()

In [ ]:
# resume_education 정렬 (resume_seq 기준)
resume_education = resume_education.sort_values(by='resume_seq')

In [ ]:
# resume_language 정렬 (resume_seq 기준)
resume_language = resume_language.sort_values(by='resume_seq')
# resume_seq, lanauge;exam_name;score
# lanauge;exam_name;score값을 넣을 컬럼 생성
resume_language['lang_exam_score'] = resume_language['language'].astype(str) + ';' + resume_language['exam_name'].astype(str) + ';' + resume_language['score'].astype(str)
# 기존 lanauge, exam_name, score 컬럼 삭제
resume_language = resume_language.drop(['language','exam_name', 'score', 'score'], axis=1)
# lang_exam_score 에 결측값이 존재하는 행 제거
resume_language = resume_language.dropna(subset=['lang_exam_score'])
# 한 유저가 여러개 자격증정보를 가지고 있으니 &로 묶기
resume_language = resume_language.groupby('resume_seq')['lang_exam_score'].apply('&'.join).reset_index()

In [ ]:
# merged_resume = resume, resume_certificate, resume_education, resume_language
merged_resume = pd.merge(resume, resume_certificate, on='resume_seq', how='left')
merged_resume = pd.merge(merged_resume, resume_education, on='resume_seq', how='left')
merged_resume = pd.merge(merged_resume, resume_language, on='resume_seq', how='left')
merged_resume

### merged_resume 탐색 & drop

In [ ]:
merged_resume.info()

In [ ]:
merged_resume.describe()

In [ ]:
merged_resume.columns

In [ ]:
(merged_resume.isnull().sum() / len(merged_resume)).sort_values()

- 결측치가 90프로 이상인 컬럼이 4개 존재
  - job_code_seq2, job_code_seq3(희망직무) : drop 해도 괜찮을 것 같다.
  - univ_sub_major(부전공) : drop 해도 괜찮을 것 같다.
  - lang_exam_score(언어,시험종류,점수) : 새로 만든 feature, 언어자격증 유무로 해야할지, drop 해야할지?

- univ_major(전공) : univ_type_seq 때문에 결측치가 많아 보인다. => drop 해도 괜찮을 것 같다.
- certificate_contents(자격증) : 자격증 유무?, 자격증 갯수, 키워드 정리할 수 있는 방법
- career_job_code(경력직무) : 키워드 정리할 수 있는 방법
- text_keyword(직무키워드) : 키워드 정리할 수 있는 방법

[other col]

- updated_date, reg_date : 기준을 나누기
- hischool_location_seq : 고등학교 지역코드 기준으로 달라질까?
- univ_type_seq1 : 대학종류로 기준
- career_month : 경력에 따른 지원기준?
- hope_salary : 희망연볼 기준?
- last_salary : 이전연봉 기준?
  - 희망연봉과 이전연봉의 관계
- 경력원에 따른 희망연봉?
- 졸업년도와 이력서 등록일에 따른 차이?

In [ ]:
col = merged_resume[['univ_type_seq1', 'univ_type_seq2',
                         'univ_transfer', 'univ_location',
                         'univ_major_type', 'univ_score']]

sns.heatmap(col.corr(),vmin=-1,vmax=1,annot=True,linewidths=0.2,cmap='coolwarm')

In [ ]:
merged_resume_drop_col = ['job_code_seq2', 'job_code_seq3', # 결측치 다수
                          'univ_major', 'univ_sub_major', 'univ_type_seq2'] # univ_type_seq1에 다 포함되어 있는 것 같다.(상광관계가 높다)
merged_resume = merged_resume.drop(merged_resume_drop_col, axis=1)
(merged_resume.isnull().sum() / len(merged_resume)).sort_values()

- lang_exam_score : 언어 자격증을 가지고 있는지 유무로 나누자
  - 언어 자격증 소유 : 1 / 미소유 : 0
- certificate_contents : 자격증을 가지고 있는지 유무로 나누자
  - 자격증 소유 : 1 / 미소유 : 0
- certificate_contents의 갯수가 중요할까?

In [ ]:
merged_resume["lang_exist"] = 0
merged_resume.loc[merged_resume["lang_exam_score"].notna(), "lang_exist"] = 1
merged_resume["lang_exist"] = merged_resume["lang_exist"].astype('category')
merged_resume.head()

In [ ]:
merged_resume["certificate_exist"] = 0
merged_resume.loc[merged_resume["certificate_contents"].notna(), "certificate_exist"] = 1
merged_resume["certificate_exist"] = merged_resume["certificate_exist"].astype('category')

merged_resume.head()

- certificate_contents  갯수

In [ ]:
merged_resume["certificate_contents"][0]

In [ ]:
merged_resume["certificate_cnt"] = 0

# if merged_resume["certificate_contents"].notna():
#   split_str = merged_resume["certificate_contents"].split(";")
#   merged_resume["certificate_cnt"] = len(split_str)

for i, row in merged_resume.iterrows():
  if pd.notna(row["certificate_contents"]):
    split_str = row["certificate_contents"].split(";")
    merged_resume.at[i, "certificate_cnt"] = len(split_str)

merged_resume[["certificate_contents", "certificate_cnt"]]

In [ ]:
# lang_exam_score, certificate_contents 삭제
drop_col = ['lang_exam_score', 'certificate_contents']
merged_resume = merged_resume.drop(drop_col, axis=1)
(merged_resume.isnull().sum() / len(merged_resume)).sort_values()

- career_month 나눠보기

In [ ]:
merged_resume['career_month'].value_counts()

In [ ]:
# 12개월을 범위로 career_month 컬럼의 값을 분류
bins = [-1, 12, 24, 36, 48, 60, 72, 84, 96, 108, merged_resume['career_month'].max()]
labels = ['1Y', '2Y', '3Y', '4Y', '5Y', '6Y', '7Y', '8Y', '9Y', '10Y']

merged_resume['career_month_range'] = pd.cut(merged_resume['career_month'], bins=bins, labels=labels)
career_month_range_counts = merged_resume['career_month_range'].value_counts().sort_index()

plt.figure(figsize=(12, 7))
career_month_range_counts.plot(kind='bar')

plt.title('merged_resume', fontsize=15)
plt.xlabel('Year', fontsize=13)
plt.ylabel('Cnt', fontsize=13)
plt.tight_layout()
plt.show()

- 1년 미만, 10년 이상이 제일 많다.
  - 년차별로?
  - 1, 3, 5, 7, 9, 10이상?
  - 1, 2-9, 10 이상?

- 1, 3, 5, 7, 9, 10이상

In [ ]:
merged_resume.loc[merged_resume['career_month_range'] == "2Y", "career_month_range"] = "3Y"
merged_resume.loc[merged_resume['career_month_range'] == "4Y", "career_month_range"] = "5Y"
merged_resume.loc[merged_resume['career_month_range'] == "6Y", "career_month_range"] = "7Y"
merged_resume.loc[merged_resume['career_month_range'] == "8Y", "career_month_range"] = "9Y"
merged_resume

In [ ]:
fillna_col = ['text_keyword', 'career_job_code']

for col in fillna_col: #for문을 돌면서 각 컬럼에 대한 최빈값으로 채움
  col_mode = merged_resume[col].mode().values[0]
  merged_resume[col].fillna(col_mode, inplace=True)

(merged_resume.isnull().sum() / len(merged_resume)).sort_values()

In [ ]:
merged_resume.info()

In [ ]:
#문자형(object type)을 LightGBM에 사용할 수 있도록 Category 형태로 형 변환
merged_resume_object = merged_resume.select_dtypes(exclude=np.number)
object_cols = list(merged_resume_object.columns)

for col in object_cols:
  merged_resume[col] = merged_resume[col].astype('category')

#형 변환 확인
merged_resume.info()

### merged_total = apply_train + merged_recruitment + merged_resume

In [ ]:
#컬럼이 너무 많아 헷갈려서 recruitment 데이터에는 'rc_' 추가
rc_columns = ['rc_' + x for x in merged_recruitment.columns]
merged_recruitment.rename(columns=dict(zip(merged_recruitment.columns, rc_columns)), inplace=True)
merged_recruitment

In [ ]:
#apply_train과 merged_recruitment를 merge한 뒤, 필요없는 rc_recruitment_seq drop
_merged_total = pd.merge(apply_train, merged_recruitment, left_on='recruitment_seq', right_on='rc_recruitment_seq', how='left')
_merged_total.drop('rc_recruitment_seq', axis = 1, inplace=True)
_merged_total

In [ ]:
#resume 관련 데이터에는 'rs_' 붙임
rs_columns = ['rs_' + x for x in merged_resume.columns]
merged_resume.rename(columns=dict(zip(merged_resume.columns, rs_columns)), inplace=True)
merged_resume

In [ ]:
#똑같은 작업 수행 뒤 최종 merged_total 데이터 생성
_merged_total = pd.merge(_merged_total, merged_resume, left_on='resume_seq', right_on='rs_resume_seq', how='left')
_merged_total.drop('rs_resume_seq', axis = 1, inplace=True)
merged_total = _merged_total
merged_total

# EDA

## merged_total 데이터 탐색

In [ ]:
merged_total.info()

In [ ]:
#결측률 확인
merged_total.isnull().sum() / len(merged_total)

### 수치형 데이터 분석 & 결측치 치환

In [ ]:
merged_total.describe()

In [ ]:
#수치형(int/float type) 데이터 추출
merged_total_number = merged_total.select_dtypes(include=np.number)

#결측치 확인
merged_total_number.isnull().sum() / len(merged_total_number)

### 문자형 데이터 분석 & 결측치 치환

In [ ]:
#문자형(object type) 데이터 추출
merged_total_object = merged_total.select_dtypes(exclude=np.number)
merged_total_object.head()

In [ ]:
#문자형 데이터만 결측율 따로보기
merged_total_object.isnull().sum() / len(merged_total_object)

## merged_total을 활용한 신규 피처 생성

### recruitment_seq 기준으로 groupby

In [ ]:
#groupby 진행해서 새로운 피처들을 여기서 뽑아낸다.
#기존의 merged_recruitment에 새롭게 생성한 피처들을 붙여서 최종 merged_recruitment를 만들어낸다.
merged_recruitment.head()

### resume_seq 기준으로 groupby

In [ ]:
#groupby 진행해서 새로운 피처들을 여기서 뽑아낸다.
#기존의 merged_recruitment에 새롭게 생성한 피처들을 붙여서 최종 merged_recruitment를 만들어낸다.
merged_resume.head()

# Negative Sampling

In [ ]:
apply_train.head()

In [ ]:
resume_pool = set(apply_train["resume_seq"].unique())
recruitment_pool = set(apply_train['recruitment_seq'].unique())

len(resume_pool), len(recruitment_pool)

#### apply_train에서 resume_seq 기준으로 지원한 공고, 지원하지 않은 공고 추출

In [ ]:
df_add_negative = apply_train.groupby(["resume_seq"])["recruitment_seq"].apply(set).reset_index().rename(columns={"recruitment_seq":"interacted_iid"})
df_add_negative

In [ ]:
# 지원하지 않은 공고
df_add_negative['negative_iid'] = df_add_negative['interacted_iid'].map(lambda x: recruitment_pool - x)
df_add_negative

In [ ]:
# 지원한 공고의 수
df_add_negative['interacted_iid_cnt'] = df_add_negative['interacted_iid'].map(lambda x: len(x))
df_add_negative

In [ ]:
# 지원한 공고 횟수만큼 지원하지 않은 회사 추출..
df_add_negative['negative_sampling'] = df_add_negative.apply(lambda row: random.sample(list(row['negative_iid']), row['interacted_iid_cnt']), axis=1)

In [ ]:
df_add_negative.head()

In [ ]:
df_add_negative[['resume_seq','interacted_iid', 'negative_sampling']].head()

In [ ]:
# 리스트인 interacted_iid 컬럼을 각각 row로 변경한다.
df_interacted = df_add_negative[['resume_seq', 'interacted_iid']].explode('interacted_iid').rename(
        columns={'interacted_iid':'recruitment_seq'}
    )
# 지원한 회사이므로 target 컬럼의 모든 값은 1
df_interacted['target'] = 1
# 인덱스 재정렬
df_interacted.reset_index(drop=True, inplace=True)

print(df_interacted.shape)
df_interacted.head()

In [ ]:
# 리스트인 negative_sampling 컬럼을 각각 row로 변경한다.
df_negatived = df_add_negative[['resume_seq', 'negative_sampling']].explode('negative_sampling').rename(
        columns={'negative_sampling':'recruitment_seq'}
    )
# 지원하지 않은 공고이므로 target은 0
df_negatived['target'] = 0
# 인덱스 재정렬
df_negatived.reset_index(drop=True, inplace=True)

print(df_negatived.shape)
df_negatived.head()

In [ ]:
# 지원한 공고 df, 지원하지 않은 공고 df를 합친다.
# axis = 0 : row로 합친다.
df_concat = pd.concat([df_interacted, df_negatived], axis=0)

print(df_concat.shape)
df_concat.head()

In [ ]:
# df_concat.sample(frac=1) : df전체를 무작위 샘플로 생성
# 인덱스 재정렬
df_shuffle = df_concat.sample(frac=1).reset_index(drop=True)

print(df_shuffle.shape)
df_shuffle.head()

In [ ]:
df_shuffle['target'].value_counts()

# df_train(학습용 데이터) 생성

In [ ]:
df_shuffle.head()

In [ ]:
merged_recruitment.head()

In [ ]:
_df_train = pd.merge(df_shuffle, merged_recruitment, left_on='recruitment_seq', right_on='rc_recruitment_seq', how='left')
df_train = pd.merge(_df_train, merged_resume, left_on='resume_seq', right_on='rs_resume_seq', how='left')
df_train.head()

In [ ]:
#필요없는 id 컬럼('recruitment_seq', 'resume_seq' 제거)
df_train_drop_col = ['recruitment_seq', 'rc_recruitment_seq', 'resume_seq', 'rs_resume_seq']
df_train.drop(df_train_drop_col, axis = 1, inplace=True)
df_train.info()

In [ ]:
df_train.isnull().sum().sum()

# modelV3 (LightGBM, base model)

## 데이터 분리

In [ ]:
df_train.info()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df_train.drop('target', axis=1)
y = df_train['target']

X_tr, X_te, y_tr, y_te = train_test_split(X, y, stratify=y, random_state=args.random_state, test_size=0.2)
X_tr = X_tr.reset_index(drop=True)
X_te = X_te.reset_index(drop=True)

X_tr.shape, X_te.shape

In [ ]:
#원본데이터 유지를 위해 copy 생성
train = X_tr.copy()
test = X_te.copy()

train.shape, test.shape

## 모델 생성

## 학습

In [ ]:
from lightgbm import LGBMClassifier, plot_importance

hp = {
    "random_state" : 42,
    "max_depth" : 2,
    "n_estimators" : 5000,
    "learning_rate": 0.01
}
model_V3 = LGBMClassifier(**hp).fit(train, y_tr)
print(f'훈련용 평가지표: {model_V3.score(train, y_tr)} / 테스트용 평가지표: {model_V3.score(test, y_te)}')

## 학습평가

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
y_tr.shape # 실제값
pred_tr = model_V3.predict(X_tr) # 예측값
pred_proba_tr = model_V3.predict_proba(X_tr)[:,1] # 예측확률

In [ ]:
fpr, tpr, thresholds = roc_curve(y_tr, pred_proba_tr)
auc_tr = auc(fpr, tpr)
print(f'auc: {auc_tr}')

In [ ]:
y_te.shape # 실제값
pred_te = model_V3.predict(X_te) # 예측값
pred_proba_te = model_V3.predict_proba(X_te)[:,1] # 예측확률

In [ ]:
# roc_curve(실제값, 예측확률값)
fpr, tpr, thresholds = roc_curve(y_te, pred_proba_te)
auc_te = auc(fpr, tpr)
print(f'auc: {auc_te}')

In [ ]:
print(f'train auc: {auc_tr} / test auc: {auc_te}')

## Best Model 해석

In [ ]:
best_model = model_V3

### Best Model 생성 및 학습

In [ ]:
y_te.shape # 실제값
pred_te = best_model.predict(X_te) # 예측값
pred_proba_te = best_model.predict_proba(X_te)[:,1] # 예측확률

In [ ]:
pred_te

In [ ]:
pred_proba_te

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
# confusion_matrix(실제값, 예측값)
# normalize="true" -> 확률값으로 변경!!
conf_mx = confusion_matrix(y_te, pred_te, normalize="true")
conf_mx

### HeatMap by Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(7,5))

# annot=True -> 수치데이터 표시
# cmap -> 히트맵 컬러정의
# linewidth -> 선 두께
sns.heatmap(conf_mx, annot=True, cmap="coolwarm", linewidth=0.5)

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# 예측

In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm

In [ ]:
all_recommendations = []

for resume in tqdm(resume_pool):
#for resume in tqdm(['U00001', 'U00002','U00003', 'U00004', 'U00005']):
    # 2. 임시 데이터프레임 생성
    _df_temp = pd.DataFrame({'resume_seq': [resume] * len(recruitment)})

    # 3. 전체 공고 seq를 붙임
    _df_temp['recruitment_seq'] = list(recruitment_pool)
    _df_temp.head()

    # 4. 임시 데이터프레임에 feature들 merge. 이후 구직자, 공고 id 제거 및 모델 학습
    _df_train = pd.merge(_df_temp, merged_recruitment, left_on='recruitment_seq', right_on='rc_recruitment_seq', how='left')
    _df_train = pd.merge(_df_train, merged_resume, left_on='resume_seq', right_on='rs_resume_seq', how='left')
    _df_train_drop_col = ['recruitment_seq', 'rc_recruitment_seq', 'resume_seq', 'rs_resume_seq']
    _df_train.drop(_df_train_drop_col, axis = 1, inplace=True)

    # 5. 예측 확률 계산
    pred_proba = best_model.predict_proba(_df_train)[:,1]

    # 6. 임시 데이터프레임에 예측 확률 추가하고 상위로 정렬
    _df_temp['prediction'] = pred_proba
    _df_temp = _df_temp.sort_values(by='prediction', ascending=False)

    # 7. 현재 구직자의 이미 지원한 공고 가져오기
    already_applied_jobs = apply_train[apply_train['resume_seq'] == resume]['recruitment_seq'].values

    # 8. 임시 데이터프레임에서 이미 지원한 공고를 제거
    _df_temp = _df_temp[~_df_temp['recruitment_seq'].isin(already_applied_jobs)]

    # 상위 5개 추천 (이미 지원한 공고를 제외한 상위 5개)
    top_recommendations = _df_temp.head(5)

    # 결과 출력 또는 활용
    all_recommendations.append(top_recommendations)

final_recommendations = pd.concat(all_recommendations, ignore_index=True)
#final_recommendations = final_recommendations[['resume_seq', 'recruitment_seq']]
final_recommendations


# 평가

In [ ]:
def recall5(answer_df, submission_df):
    """
    Calculate recall@5 for given dataframes.

    Parameters:
    - answer_df: DataFrame containing the ground truth
    - submission_df: DataFrame containing the predictions

    Returns:
    - recall: Recall@5 value
    """

    primary_col = answer_df.columns[0]
    secondary_col = answer_df.columns[1]

    # submission의 예측이 각각 5개인지 확인
    prediction_counts = submission_df.groupby(primary_col).size()
    if not all(prediction_counts == 5):
        raise ValueError(f"Each {primary_col} should have exactly 5 {secondary_col} predictions.")


    # submission의 예측된 값들에 null값이 있는지 확인
    if submission_df[secondary_col].isnull().any():
        raise ValueError(f"Predicted {secondary_col} contains NULL values.")

    # 예측값에 중복이 있는지 확인
    duplicated_preds = submission_df.groupby(primary_col).apply(lambda x: x[secondary_col].duplicated().any())
    if duplicated_preds.any():
        raise ValueError(f"Predicted {secondary_col} contains duplicates for some {primary_col}.")


    # primary_col 즉 resume_seq가 양측에 있는지 확인 후 남김
    submission_df = submission_df[submission_df[primary_col].isin(answer_df[primary_col])]

    # For each primary_col, get the top 5 predicted secondary_col values
    top_5_preds = submission_df.groupby(primary_col).apply(lambda x: x[secondary_col].head(5).tolist()).to_dict()

    # Convert the answer_df to a dictionary for easier lookup
    true_dict = answer_df.groupby(primary_col).apply(lambda x: x[secondary_col].tolist()).to_dict()


    individual_recalls = []
    for key, val in true_dict.items():
        if key in top_5_preds:
            correct_matches = len(set(true_dict[key]) & set(top_5_preds[key]))
            individual_recall = correct_matches / min(len(val), 5) # 공정한 평가를 가능하게 위하여 분모(k)를 'min(len(val), 5)' 로 설정함
            individual_recalls.append(individual_recall)


    recall = np.mean(individual_recalls)
    return recall

# Result 저장 (Submission)
- Dictionary List로 sort해서 best model select

In [ ]:
top_recommendations = pd.DataFrame(final_recommendations, columns=['resume_seq', 'recruitment_seq'])
top_recommendations.to_csv(args.submission_path, index=False)